# Molecular dynamics: Protein-ligand complex Tutorial

###### Gutavo Enrique Olivos Ramiez
###### Laboratorio de Bioinformática y Biología Molecular, Laboratorios de Investigación y Desarrollo, Universidad Peruana Cayetano Heredia, Lima, Perú.

<td class="first">
    <p style="text-align: justify;font-family: Comic;color:black;font-size:1.1em; font-weight: bold">Software requirements:</p>
<p style="text-align: justify;font-family: Comic;color:black;font-size:1.0em">- Gromacs 2018.3</p>
<p style="text-align: justify;font-family: Comic;color:black;font-size:1.0em">- Grace 5.1.25</p>
    <p style="text-align: justify;font-family: Comic;color:black;font-size:1.0em">- VMD 1.9.3</p>
<p style="text-align: justify;font-family: Comic;color:black;font-size:1.1em; font-weight: bold">Required input files: </p>
<p style="text-align: justify;font-family: Comic;color:black;font-size:1.0em">- receptor.pdb </p>
<p style="text-align: justify;font-family: Comic;color:black;font-size:1.0em">- ligand.pdb </p>
</td>
<td class="third">
<p style="text-align: justify;font-family: Comic;color:black;font-size:1.1em; font-weight: bold">Scripts </p>
<p style="text-align: justify;font-family: Comic;color:black;font-size:1.0em">- ions.mdp</p>
<p style="text-align: justify;font-family: Comic;color:black;font-size:1.0em">- em.mdp</p>
<p style="text-align: justify;font-family: Comic;color:black;font-size:1.0em">- nvt.mdp</p>
<p style="text-align: justify;font-family: Comic;color:black;font-size:1.0em">- npt.mdp</p>
<p style="text-align: justify;font-family: Comic;color:black;font-size:1.0em">- md.mdp</p>
</td>

### Step 1: construyendo la topología de la proteína y ligando

In [ ]:
%%bash 
gmx="/home/pitts/Documents/gromacs-2018.3/build/bin/./gmx_mpi"
echo -e '10' | $gmx pdb2gmx -f receptor.pdb -o receptor.gro -water spce
# El comando pdb2gmx permite generar la topologia del receptor.

In [ ]:
!gedit ligand.pdb

In [ ]:
import webbrowser
url = 'http://prodrg1.dyndns.org/'
webbrowser.open(url)
# La topologia del ligando debe ser construida en un servidor externo, dependiendo del FF seleccionado. 

In [ ]:
%%bash
mkdir ligand_topology && mv *.tgz ligand_topology/
cd ligand_topology && tar -xvzf *.tgz && cd ..
cp ligand_topology/DRGAPH.GRO .
mv DRGAPH.GRO ligand.gro
cp ligand_topology/DRGGMX.ITP .
mv DRGGMX.ITP unl.itp

#### Nota: Hasta este paso habremos generado 3 archivos de la proteína (.gro, .itp, .top) y dos más del ligando (.gro, .itp).

### Step 2: construyendo el complejo

In [ ]:
%%bash
gedit receptor.gro ligand.gro complex.gro

### Step 3: construyendo la topologia del complejo

In [ ]:
#En este procedimiento incluiremos los parametros del ligando a la topologia.

; Include Position restraint file
#ifdef POSRES
#include "posre.itp"
#endif

; Include ligand topology
#include "unl.itp"

; Include water topology
#include "./charmm36-mar2019.ff/tip3p.itp"

In [ ]:
#Finalmente, incluiremos lo siguiente en la directiva de las moleculas:

[ molecules ]
; Compound        #mols
Protein_chain_A     1
UNL                 1

In [ ]:
%%bash
gedit topol.top

### Step 4: solvatando el sistema

In [ ]:
%%bash 
gmx="/home/pitts/Documents/gromacs-2018.3/build/bin/./gmx_mpi"
$gmx editconf -f complex.gro -o newbox.gro -bt cubic -d 1.0

In [ ]:
!vmd newbox.gro

In [ ]:
%%bash 
gmx="/home/pitts/Documents/gromacs-2018.3/build/bin/./gmx_mpi"
$gmx solvate -cp newbox.gro -cs spc216.gro -p topol.top -o solv.gro
# En esta parte se ha adicionado agua y credado una cubo de simulación.

In [ ]:
!vmd solv.gro

### Step 5: agregando iones

In [ ]:
import webbrowser
url = 'https://drive.google.com/file/d/1Xa4uMJUMqhoTD0YWQNO9gp2n5GLR9N1Y/view?usp=sharing'
webbrowser.open(url)
# En esta parte descagaremos los scripts necesarios para continuar con la dinámica molecular

In [ ]:
%%bash
unzip md_files.zip 

In [ ]:
%%bash 
gmx="/home/pitts/Documents/gromacs-2018.3/build/bin/./gmx_mpi"
$gmx grompp -f ions.mdp -c solv.gro -p topol.top -o ions.tpr

In [ ]:
%%bash 
gmx="/home/pitts/Documents/gromacs-2018.3/build/bin/./gmx_mpi"
echo -e '15' | $gmx genion -s ions.tpr -o solv_ions.gro -p topol.top -pname NA -nname CL -neutral

### Step 6: minimizando el sistema

In [ ]:
%%bash 
gmx="/home/pitts/Documents/gromacs-2018.3/build/bin/./gmx_mpi"
$gmx grompp -f em.mdp -c solv_ions.gro -p topol.top -o em.tpr

In [ ]:
%%bash 
gmx="/home/pitts/Documents/gromacs-2018.3/build/bin/./gmx_mpi"
$gmx mdrun -v -nb gpu -deffnm em

### Step 7: restringiendo el ligando

In [ ]:
%%bash
gmx="/home/pitts/Documents/gromacs-2018.3/build/bin/./gmx_mpi"
cat << EOF | $gmx make_ndx -f ligand.gro -o index_unl.ndx
0 & ! aH*
q
EOF

In [ ]:
%%bash 
gmx="/home/pitts/Documents/gromacs-2018.3/build/bin/./gmx_mpi"
echo -e '3' | $gmx genrestr -f ligand.gro -n index_unl.ndx -o posre_unl.itp -fc 1000 1000 1000

In [ ]:
#Ahora incluimos esta informacion en la topologia.

; Include Position restraint file
#ifdef POSRES
#include "posre.itp"
#endif

; Include ligand topology
#include "unl.itp"

; Ligand position restraints
#ifdef POSRES
#include "posre_unl.itp"
#endif

; Include water topology
#include "./charmm36-mar2019.ff/tip3p.itp"

In [ ]:
%%bash
gedit topol.top

### Step 8: equilibrando la temperatura

In [ ]:
%%bash 
gmx="/home/pitts/Documents/gromacs-2018.3/build/bin/./gmx_mpi"
cat << EOF | $gmx make_ndx -f em.gro -o index.ndx
1 | 13
q
EOF

In [ ]:
%%bash
gmx="/home/pitts/Documents/gromacs-2018.3/build/bin/./gmx_mpi"
$gmx grompp -f nvt.mdp -c em.gro -r em.gro -p topol.top -n index.ndx -o nvt.tpr

In [ ]:
%%bash 
gmx="/home/pitts/Documents/gromacs-2018.3/build/bin/./gmx_mpi"
$gmx mdrun -v -nb gpu -deffnm nvt

### Step 9: equilibrando la presión

In [ ]:
%%bash 
gmx="/home/pitts/Documents/gromacs-2018.3/build/bin/./gmx_mpi"
$gmx grompp -f npt.mdp -c nvt.gro -t nvt.cpt -r nvt.gro -p topol.top -n index.ndx -o npt.tpr

In [ ]:
%%bash 
gmx="/home/pitts/Documents/gromacs-2018.3/build/bin/./gmx_mpi"
$gmx mdrun -v -nb gpu -deffnm npt

### Step 10: ejecuntando la dinámica molecular

In [ ]:
%%bash
gedit md.mdp
#Aqui hay que modificar los steps para establecer el tiempo de la simulación.

In [ ]:
%%bash 
gmx="/home/pitts/Documents/gromacs-2018.3/build/bin/./gmx_mpi"
$gmx grompp -f md.mdp -c npt.gro -t npt.cpt -p topol.top -n index.ndx -o md_0_10.tpr

In [ ]:
%%bash 
gmx="/home/pitts/Documents/gromacs-2018.3/build/bin/./gmx_mpi"
$gmx mdrun -v -nb gpu -deffnm md_0_10

### Step 11: analizando los resultados

In [ ]:
%%bash 
gmx="/home/pitts/Documents/gromacs-2018.3/build/bin/./gmx_mpi"
cat << EOF | $gmx rms -s md_0_10.tpr -f md_0_10.xtc -o rmsd2.xvg -tu ns
4
4
EOF
# RMSD plot

In [ ]:
%%bash
xmgrace rmsd2.xvg

In [ ]:
%%bash 
gmx="/home/pitts/Documents/gromacs-2018.3/build/bin/./gmx_mpi"
cat << EOF | $gmx rmsf -s md_0_10.tpr -f md_0_10.xtc -res -o rmsf.xvg
4
4
EOF
# RMSF plot

In [ ]:
%%bash
xmgrace rmsf.xvg

In [ ]:
%%bash 
gmx="/home/pitts/Documents/gromacs-2018.3/build/bin/./gmx_mpi"
$gm trjconv -s topol.tpr -f md_0_10.xtc -dt 100 -o trj.pdb
# Trajectorias en pdb

In [ ]:
!vmd trj.pdb

###### Felicidades, hemos concluido la práctica!!!